<a href="https://colab.research.google.com/github/leonardoub/SCRIPT_PALERMO/blob/master/prova_hyperparameter_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install -U keras-tuner

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [0]:
!rm -r my_dir

#Load data

In [4]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
train_dataset_path = '/gdrive/My Drive/AIM_PA/database_training2.csv'
test_dataset_path = '/gdrive/My Drive/AIM_PA/database_nostro_without_nan.csv'

In [0]:
df_train = pd.read_csv(train_dataset_path)
df_test = pd.read_csv(test_dataset_path)

In [0]:
df_train.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_test.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_train.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)
df_test.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)

In [0]:
train_data = df_train.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
test_data = df_test.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
train_labels = df_train.Histology

In [0]:
test_labels = df_test.Histology

##Z score dei dati

In [0]:
mean = train_data.mean(axis=0)
train_data_stand = train_data - mean
std = train_data.std(axis=0)
train_data_stand /= std

In [0]:
test_data_stand = test_data - mean
test_data_stand /= std

##Vettorizzare i label

In [0]:
word_index={'adenocarcinoma':0, 'large cell':1, 'squamous cell carcinoma':2}

In [0]:
train_labels_dec = [word_index[label] for label in train_labels]

In [0]:
test_labels_dec = [word_index[label] for label in test_labels]

In [19]:
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [0]:
one_hot_train_labels = to_categorical(train_labels_dec)
one_hot_test_labels = to_categorical(test_labels_dec)

#PCA

In [0]:
from sklearn.decomposition import PCA

In [0]:
pca = PCA(n_components=0.9, svd_solver='full')

In [23]:
pca.fit(train_data_stand)

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [0]:
train_data_stand_pca = pca.transform(train_data_stand)
test_data_stand_pca = pca.transform(test_data_stand)

In [25]:
train_data_stand_pca.shape

(131, 9)

#Building Network

In [0]:
from tensorflow import keras

In [0]:
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence

In [0]:
from tensorflow.keras import layers

In [0]:
from tensorflow.keras import regularizers

In [0]:
from tensorflow.keras.optimizers import SGD

In [0]:
from tensorflow.keras.optimizers import Adam

In [0]:
from kerastuner.tuners import RandomSearch, Hyperband

In [0]:
def build_model(hp):
  model = keras.models.Sequential()
  activation = hp.Choice('activation', ['softmax', 'softplus', 'softsign',
                              'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'])
  
  weight_init = hp.Choice('weight_init', 
                            ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'])
  
  drop_rate = hp.Choice('drop_rate', [0.0, 0.1, 0.2, 0.3,
                              0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
  
  model.add(layers.Dense(units=(hp.Int('units', min_value=3, max_value=8, step=1)), 
                         activation=activation, kernel_initializer=weight_init, input_shape=(9,)))
  
  model.add(layers.Dropout(rate=drop_rate))

  model.add(layers.Dense(3, activation='softmax'))
  #sgd_1 = SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)
 
  #optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])

  lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
  momentum = hp.Choice('momentum', [0.0, 0.2, 0.4, 0.6, 0.8, 0.9])
  
  model.compile(optimizer=SDG(learning_rate=lr, momentum=momentum), 
                loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#Stratified k-fold

This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

##Prova Train Test Split

In [0]:
#from sklearn.model_selection import train_test_split


In [0]:
#X_train, X_val, y_train, y_val = train_test_split(train_data_stand_pca, train_labels_dec,
#                                                    stratify=train_labels_dec,
#                                                    test_size=0.20,
#                                                    random_state=30)


In [0]:
#y_train.count(2)

con il parametro stratify si conservano le proporzioni dei label, però è meglio fare anche il K-Fold

##Stratifield K-fold

In [0]:
from sklearn.model_selection import StratifiedKFold

In [36]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
skf.get_n_splits(train_data_stand_pca, train_labels_dec)

3

In [37]:
for train_index, test_index in skf.split(train_data_stand_pca, train_labels_dec):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [  0   1   4   5   8   9  11  12  14  15  16  17  19  20  22  23  24  25
  27  29  30  33  34  36  37  38  39  40  41  42  44  45  46  48  51  52
  53  56  57  58  59  60  62  63  65  66  67  69  72  76  77  78  79  80
  81  83  84  85  87  88  89  90  92  96  97  98 100 101 102 103 104 105
 107 109 110 111 113 115 117 120 121 122 124 125 127 128] TEST: [  2   3   6   7  10  13  18  21  26  28  31  32  35  43  47  49  50  54
  55  61  64  68  70  71  73  74  75  82  86  91  93  94  95  99 106 108
 112 114 116 118 119 123 126 129 130]
TRAIN: [  2   3   5   6   7   8   9  10  11  12  13  18  20  21  25  26  27  28
  29  30  31  32  34  35  36  38  39  43  44  45  46  47  48  49  50  53
  54  55  57  58  61  63  64  65  66  68  70  71  73  74  75  76  78  82
  84  85  86  87  90  91  92  93  94  95  96  99 100 101 102 105 106 108
 109 111 112 114 115 116 118 119 122 123 124 125 126 127 129 130] TEST: [  0   1   4  14  15  16  17  19  22  23  24  33  37  40  41  42  51  52
  56  59 

In [38]:
train_labels_dec[125]

2

In [0]:
from keras.utils import to_categorical


In [0]:
#for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
#  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
#  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])

#  val_data = np.array([train_data_stand_pca[i] for i in val_index])
#  val_targets = np.array([train_labels_dec[i] for i in val_index])

#  one_hot_partial_train_targets = to_categorical(partial_train_targets)
#  one_hot_val_targets = to_categorical(val_targets)

#  model = build_model()
#  model.fit(partial_train_data, one_hot_partial_train_targets, epochs = num_epochs, batch_size=1)

#  val_loss, val_accuracy = model.evaluate(val_data, one_hot_val_targets)
#  all_scores.append(val_accuracy)
#I parametri per la valutazione vengono calcolati una volta per ogni k-fold, per ogni set di validazione, quindi k volte

C'è un problema: keras.utils.to_categorical produces a one-hot encoded class vector, i.e. the multilabel-indicator mentioned in the error message. StratifiedKFold is not designed to work with such input; i.e. your y must be a 1-D array of your class labels.
Essentially, what you have to do is simply to invert the order of the operations: split first (using your intial y_train), and convert to_categorical afterwards.

#Keras tuner RandomSearch

In [41]:
num_epochs = 50
all_acc_histories = []
all_loss_histories = []
all_val_acc_histories = []
all_val_loss_histories = []

for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])
  
  val_data = np.array([train_data_stand_pca[i] for i in val_index])
  val_targets = np.array([train_labels_dec[i] for i in val_index])

  one_hot_partial_train_targets = to_categorical(partial_train_targets)
  one_hot_val_targets = to_categorical(val_targets)

 
  tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=20, 
                       executions_per_trial=5, directory='/content/my_dir', project_name='RandomSearch')
  
  tuner.search_space_summary()

  tuner.search(partial_train_data, one_hot_partial_train_targets, validation_data=(val_data, one_hot_val_targets), 
                      epochs=num_epochs, batch_size=10)
  


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-33-d3f273619da5>", line 25, in build_model
    model.compile(optimizer=SDG(learning_rate=lr, momentum=momentum),
NameError: name 'SDG' is not defined


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-33-d3f273619da5>", line 25, in build_model
    model.compile(optimizer=SDG(learning_rate=lr, momentum=momentum),
NameError: name 'SDG' is not defined


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-33-d3f273619da5>", line 25, in build_model
    model.compile(optimizer=SDG(learning_rate=lr, momentum=momentum),
NameError: name 'SDG' is not defined


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-33-d3f273619da5>", line 25, in build_model
    model.compile(optimizer=SDG(learning_rate=lr, momentum=momentum),
NameError: name 'SDG' is not defined


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-33-d3f273619da5>", line 25, in build_model
    model.compile(optimizer=SDG(learning_rate=lr, momentum=momentum),
NameError: name 'SDG' is not defined


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-33-d3f273619da5>", line 25, in build_model
    model.compile(optimizer=SDG(learning_rate=lr, momentum=momentum),
NameError: name 'SDG' is not defined


RuntimeError: ignored

In [0]:
tuner.search_space_summary()


In [0]:
tuner.results_summary()

In [0]:
random_params=tuner.get_best_hyperparameters()[0]

In [0]:
random_model = tuner.hypermodel.build(random_params)

In [0]:
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [0]:
best_model = tuner.get_best_models()[0]

##Train the best model RandomSearch

In [0]:
num_epochs = 50
all_acc_histories_RS = []
all_loss_histories_RS = []
all_val_acc_histories_RS = []
all_val_loss_histories_RS = []

for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])
  
  val_data = np.array([train_data_stand_pca[i] for i in val_index])
  val_targets = np.array([train_labels_dec[i] for i in val_index])

  one_hot_partial_train_targets = to_categorical(partial_train_targets)
  one_hot_val_targets = to_categorical(val_targets)

 
  history_RS = random_model.fit(partial_train_data, one_hot_partial_train_targets, validation_data=(val_data, one_hot_val_targets), 
                      epochs=num_epochs, batch_size=10)
  
  acc_history_RS = history_RS.history['accuracy']
  all_acc_histories_RS.append(acc_history_RS)

  loss_history_RS = history_RS.history['loss']
  all_loss_histories_RS.append(loss_history_RS)

  acc_val_history_RS = history_RS.history['val_accuracy']
  all_val_acc_histories_RS.append(acc_val_history_RS)

  loss_val_history_RS = history_RS.history['val_loss']
  all_val_loss_histories_RS.append(loss_val_history_RS)
  

#I parametri per la valutazione vengono calcolati per ogni epoca, quindi num_epochs volte. 
#Il tutto viene ripetuto un numero di volte pari a n_splits.
#Si ottiene una lista con n_splits elementi ciascuno dei quali è una lista lunga num_epochs,
#ogni elemento può essere uno fra questi: dict_keys(['val_loss', 'val_acc', 'loss', 'acc']) 

In [0]:
history_dict = history_RS.history

In [0]:
history_dict.keys()

#Keras tuner Hyperband

In [0]:
num_epochs = 50

for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])
  
  val_data = np.array([train_data_stand_pca[i] for i in val_index])
  val_targets = np.array([train_labels_dec[i] for i in val_index])

  one_hot_partial_train_targets = to_categorical(partial_train_targets)
  one_hot_val_targets = to_categorical(val_targets)

 
  tuner_Hb = Hyperband(build_model, objective='val_accuracy', factor=3, 
                       max_epochs=10, directory='/content/my_dir', project_name='Hyperband')
  
  tuner_Hb.search_space_summary()

  tuner_Hb.search(partial_train_data, one_hot_partial_train_targets, 
                  validation_data=(val_data, one_hot_val_targets))
  

In [0]:
tuner_Hb.search_space_summary()


In [0]:
tuner_Hb.results_summary()

In [0]:
tuner_Hb.get_best_hyperparameters()

In [0]:
tuner_Hb.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [0]:
Hb_model = tuner_Hb.get_best_models()[0]

##Train the best model Hyperband

In [0]:
num_epochs = 50

all_acc_histories_Hb = []
all_loss_histories_Hb = []
all_val_acc_histories_Hb = []
all_val_loss_histories_Hb = []

for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])
  
  val_data = np.array([train_data_stand_pca[i] for i in val_index])
  val_targets = np.array([train_labels_dec[i] for i in val_index])

  one_hot_partial_train_targets = to_categorical(partial_train_targets)
  one_hot_val_targets = to_categorical(val_targets)

  history_Hb = Hb_model.fit(partial_train_data, one_hot_partial_train_targets, validation_data=(val_data, one_hot_val_targets), 
                      epochs=num_epochs, batch_size=10)
  
  acc_history_Hb = history_Hb.history['accuracy']
  all_acc_histories_Hb.append(acc_history_Hb)

  loss_history_Hb = history_Hb.history['loss']
  all_loss_histories_Hb.append(loss_history_Hb)

  acc_val_history_Hb = history_Hb.history['val_accuracy']
  all_val_acc_histories_Hb.append(acc_val_history_Hb)

  loss_val_history_Hb = history_Hb.history['val_loss']
  all_val_loss_histories_Hb.append(loss_val_history_Hb)
   

#I parametri per la valutazione vengono calcolati per ogni epoca, quindi num_epochs volte. 
#Il tutto viene ripetuto un numero di volte pari a n_splits.
#Si ottiene una lista con n_splits elementi ciascuno dei quali è una lista lunga num_epochs,
#ogni elemento può essere uno fra questi: dict_keys(['val_loss', 'val_acc', 'loss', 'acc']) 

#Plot RandomSearch

##Plotting training and validation loss RandomSearch

---



In [0]:
num_epochs=50

In [0]:
import matplotlib.pyplot as plt

In [0]:
epochs = range(1, num_epochs+1)

In [0]:
average_acc_history_RS = [np.mean([x[i] for x in all_acc_histories_RS]) for i in range(num_epochs)]
average_loss_history_RS = [np.mean([x[i] for x in all_loss_histories_RS]) for i in range(num_epochs)]
average_val_acc_history_RS = [np.mean([x[i] for x in all_val_acc_histories_RS]) for i in range(num_epochs)]
average_val_loss_history_RS = [np.mean([x[i] for x in all_val_loss_histories_RS]) for i in range(num_epochs)]
#media per epoca degli score ottenuti per tutte le k-fold
#per ogni k-fold di fanno num_epoch epoche, la media viene fatta prendendo gli score di tutti i k-fold relativi ad una data epoca,
#e si fa questo per tutte le epoche

In [0]:
plt.plot(epochs, average_loss_history_RS, 'bo', label='training loss')
plt.plot(epochs, average_val_loss_history_RS, 'b', label='validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

##Plotting train and validation accuracy RandomSearch

In [0]:
plt.plot(epochs, average_acc_history_RS, 'bo', label='Training accuracy')
plt.plot(epochs, average_val_acc_history_RS, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend() 

#Plot Hyperband

##Plotting training and validation loss Hyperband

---



In [0]:
import matplotlib.pyplot as plt

In [0]:
epochs = range(1, num_epochs+1)

In [0]:
average_acc_history_Hb = [np.mean([x[i] for x in all_acc_histories_Hb]) for i in range(num_epochs)]
average_loss_history_Hb = [np.mean([x[i] for x in all_loss_histories_Hb]) for i in range(num_epochs)]
average_val_acc_history_Hb = [np.mean([x[i] for x in all_val_acc_histories_Hb]) for i in range(num_epochs)]
average_val_loss_history_Hb = [np.mean([x[i] for x in all_val_loss_histories_Hb]) for i in range(num_epochs)]
#media per epoca degli score ottenuti per tutte le k-fold
#per ogni k-fold di fanno num_epoch epoche, la media viene fatta prendendo gli score di tutti i k-fold relativi ad una data epoca,
#e si fa questo per tutte le epoche

In [0]:
plt.plot(epochs, average_loss_history_Hb, 'bo', label='training loss')
plt.plot(epochs, average_val_loss_history_Hb, 'b', label='validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

##Plotting train and validation accuracy Hyperband

In [0]:
plt.plot(epochs, average_acc_history_Hb, 'bo', label='Training accuracy')
plt.plot(epochs, average_val_acc_history_Hb, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend() 